In [2]:
#import tflite_runtime.interpreter as tflite
import tensorflow as tf
import datetime
import numpy as np
import cv2
import time

In [3]:
interpreter = tf.lite.Interpreter(model_path="posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite")
interpreter.allocate_tensors()

In [4]:
def get_output_tensor(interpreter, index):

    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    
    return tensor

In [5]:
def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

In [6]:
def getInputImageDimensions(image):
    
    return image.shape[0],image.shape[1]

In [7]:
def sigmoid(x,k):
    
     return 1 / (1 + np.exp(-x/k));

In [8]:
def getPointsConfidence(heatmap_scores,heatmap_coords):
    
    numKeyPoints = heatmap_coords.shape[0]
    result = []
    
    for keypoint in range(0, numKeyPoints):
        y = heatmap_coords[keypoint][0]
        x = heatmap_coords[keypoint][1]
        result.append(heatmap_scores[y][x][keypoint])
    
    result = np.array(result)
    
    return result

In [9]:
def getOffsetPoints(heatmapCoords,offset,img_height, img_width):
    
    numKeyPoints = heatmapCoords.shape[0]
    result =[]
    
    for keypoint in range(0,numKeyPoints):
        heatmapY = heatmapCoords[keypoint][0]
        heatmapX = heatmapCoords[keypoint][1]
        y = int((heatmapY*img_height)/8 + offset[heatmapY][heatmapX][keypoint])
        x = int((heatmapX*img_width)/8 + offset[heatmapY][heatmapX][keypoint+ numKeyPoints])        
        result.append((y,x))
    
    result = np.array(result)  #shape(17,2)  
    
    return result

In [10]:
def detect_poses(interpreter, image,input_resolution):
    
    flip_horizontal = False
    
    img_height, img_width = getInputImageDimensions(image)
    resized = cv2.resize(image,(input_resolution[0], input_resolution[1]))
    
    #model posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite runs from -1.0 ... +1.0
    resized = (resized -127.5)/127.5
    
    set_input_tensor(interpreter, resized)
    interpreter.invoke()
    
    # Get all output details
    heatmap = get_output_tensor(interpreter,0)   #shape: (9,9,17)
    offset = get_output_tensor(interpreter,1)    #shape: (9,9,34)
    #heatmap = sigmoid(heatmap,1)                       
    
    height, width, depth = heatmap.shape
    reshaped = heatmap.reshape(height*width,depth)   #shape: (81,17)
    coords = np.argmax(reshaped,0)
    ycoords = coords//width
    xcoords = coords%width

    heatmap_coords=[]
    for i in range(0,17):
        heatmap_coords.append((ycoords[i],xcoords[i]))
    heatmap_coords = np.array(heatmap_coords) #shape: (17,2)
    
    keypoint_confidence = getPointsConfidence(heatmap,heatmap_coords)
    keypoint_confidence = sigmoid(keypoint_confidence,1)
    keypoints =  getOffsetPoints(heatmap_coords,offset,img_height, img_width)
    
    result_keypoints =[]
    for i in range(0,len(keypoints)):
            result_keypoints.append((int(keypoints[i][1]),int(keypoints[i][0])))
    
    return result_keypoints,keypoint_confidence       

In [11]:
def drawPoint(img,center,radius):

    thickness =-1
    color = (0, 255, 0)
    img = cv2.circle(img, center, radius, color, thickness)   

In [12]:
confidence_threshold = 0.8;    

def drawKeypoints(img, k_coords, k_conf):
    
    n =len(k_coords)
    
    for i in range(0,n):
        if(k_conf[i]> confidence_threshold):
            drawPoint(img,k_coords[i],5)

In [13]:
def drawLine(img,start_point, end_point, thickness):

    color = (0, 255, 0)
    img = cv2.line(img, start_point, end_point, color, thickness)

In [14]:
confidence_threshold = 0.8    

def drawSkeleton(img, k_coords,k_conf):
    
    if(k_conf[ 5]>confidence_threshold):
        if(k_conf[6]>confidence_threshold):
            drawLine(img,k_coords[ 5],k_coords[ 6],2)
        if(k_conf[7]>confidence_threshold):
            drawLine(img,k_coords[ 5],k_coords[ 7],2)
        if(k_conf[11]>confidence_threshold):
            drawLine(img,k_coords[ 5],k_coords[11],2)
    
    if(k_conf[ 6]>confidence_threshold):
        if(k_conf[8]>confidence_threshold):  drawLine(img,k_coords[6],k_coords[ 8],2)
        if(k_conf[12]>confidence_threshold): drawLine(img,k_coords[6],k_coords[12],2)
    
    if(k_conf[7]>confidence_threshold):
        if(k_conf[9]>confidence_threshold):  drawLine(img,k_coords[7],k_coords[9],2)
    
    if(k_conf[ 8]>confidence_threshold):
        if(k_conf[10]>confidence_threshold): drawLine(img,k_coords[ 8],k_coords[10],2);
    
    if(k_conf[11]>confidence_threshold):
        if(k_conf[12]>confidence_threshold): drawLine(img,k_coords[11],k_coords[12],2);
        if(k_conf[13]>confidence_threshold): drawLine(img,k_coords[11],k_coords[13],2);
    
    if(k_conf[13]>confidence_threshold):
        if(k_conf[15]>confidence_threshold): drawLine(img,k_coords[13],k_coords[15],2)
    
    if(k_conf[14]>confidence_threshold):
        if(k_conf[12]>confidence_threshold): drawLine(img,k_coords[14],k_coords[12],2)
        if(k_conf[16]>confidence_threshold): drawLine(img,k_coords[14],k_coords[16],2)
    

In [16]:
def Pose_image():
    _, input_height, input_width,channels = interpreter.get_input_details()[0]['shape']
    
    
    image = cv2.imread('image_2.jpg',1)
    keypoints, keypoint_confidence = detect_poses(interpreter,image,(input_width,input_height))
    
    #print(keypoint_confidence)
    drawKeypoints(image,keypoints,keypoint_confidence)
    drawSkeleton(image,keypoints,keypoint_confidence)
    cv2.imshow("Pose",image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
Gait()    

[0.99420434 0.991954   0.89997613 0.9356845  0.10531113 0.9914641
 0.9810146  0.9926959  0.9692133  0.9068489  0.90729076 0.9932314
 0.982566   0.92077124 0.895883   0.8823035  0.9636957 ]


In [14]:
def Pose_video():
    
    _, input_height, input_width,channels = interpreter.get_input_details()[0]['shape']
    vid = cv2.VideoCapture(0)
    
    fps=0
    
    while(True):
        fps_start_time = time.time()
        
        ret, image = vid.read() 
        
        keypoints, keypoint_confidence = detect_poses(interpreter,image,(input_height, input_width))
        drawKeypoints(image,keypoints,keypoint_confidence)
        drawSkeleton(image,keypoints,keypoint_confidence)
        
        time_diff = time.time() - fps_start_time
        
        if(time_diff ==0):
            fps =0
        else:
            fps = 1/ time_diff
            
        fps_text = "FPS {:.2f}".format(fps)
        cv2.putText(image,fps_text,(5,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 0, 0),1)
        cv2.imshow('Pose', image) 
    
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
  
    vid.release() 
    cv2.destroyAllWindows()

Gait_video()